In [1]:
import sys

import matplotlib.gridspec as gridspec
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
from PIL import Image

sys.path.append("../src")
from data_utils import UnlearningDataLoader
import subprocess
import time
import warnings
from datetime import datetime

import mlflow
import torch
import torch.nn as nn
from torch.optim import SGD, Adam
from torch.optim.lr_scheduler import LambdaLR

from config import set_config
from data_utils import UnlearningDataLoader
from eval import (
    compute_accuracy,
    get_forgetting_rate,
    get_js_div,
    get_l2_params_distance,
    mia,
)
from mlflow_utils import mlflow_tracking_uri
from models import VGG19, AllCNN, ResNet18, ViT
from seed import set_seed
from sklearn.decomposition import PCA

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

/home/cspartalis/anaconda3/envs/MaUn/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from torchvision import transforms
from torch.utils.data import DataLoader

In [3]:
def calculate_mean_std(loader):
    mean = 0.0
    for images, _ in loader:
        batch_samples = images.size(
            0
        )  # batch size (the last batch can have smaller size!)
        images = images.view(batch_samples, images.size(1), -1)
        mean += images.mean(2).sum(0)
    mean /= len(loader.dataset)

    var = 0.0
    for images, _ in loader:
        batch_samples = images.size(0)
        images = images.view(batch_samples, images.size(1), -1)
        var += ((images - mean.unsqueeze(1)) ** 2).sum([0, 2])
    std = torch.sqrt(var / (len(loader.dataset) * 224 * 224))

    return mean, std

In [4]:
# UDL = UnlearningDataLoader("tissuemnist", 128)
# dl, _ = UDL.load_data()

In [5]:
# train_mean, train_std = calculate_mean_std(dl["train"])
# val_mean, val_std = calculate_mean_std(dl["val"])
# test_mean, test_std = calculate_mean_std(dl["test"])

In [6]:
# print(f"train_mean: {round(train_mean.item(),4)}")
# print(f"train_std: {round(train_std.item(),2)}")
# print(f"val_mean: {round(val_mean.item(),2)}")
# print(f"val_std: {round(val_std.item(),2)}")
# print(f"test_mean: {round(test_mean.item(),2)}")
# print(f"test_std: {round(test_std.item(),2)}")

In [7]:
UDL = UnlearningDataLoader("pneumoniamnist", 128)
dl, _ = UDL.load_data()

In [8]:
train_mean, train_std = calculate_mean_std(dl["train"])
val_mean, val_std = calculate_mean_std(dl["val"])
test_mean, test_std = calculate_mean_std(dl["test"])

In [9]:
print(f"train_mean: {round(train_mean.item(),4)}")
print(f"train_std: {round(train_std.item(),2)}")
print(f"val_mean: {round(val_mean.item(),2)}")
print(f"val_std: {round(val_std.item(),2)}")
print(f"test_mean: {round(test_mean.item(),2)}")
print(f"test_std: {round(test_std.item(),2)}")

train_mean: 0.5719
train_std: 0.02
val_mean: 0.57
val_std: 0.02
test_mean: 0.56
test_std: 0.02
